<a href="https://colab.research.google.com/github/Sahilk5/ASB/blob/main/ABL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [269]:
print("Welcome to my Programming Language")

Welcome to my Programming Language


In [270]:
import keyword
import re

In [271]:
# Environment name storage

class Environment:
  # record - Store of all local variables
  def __init__(self, record = {}):
    self.record = record

  # Add name entry in record
  def define(self, name, value):
    self.record[name] = value
    return value
  def lookupVar(self, name):
    if name in self.record:
      return self.record[name]
    else:
      raise NameError("No variable named {} found".format(name))

In [272]:
class Abl:
    # Create an Abl instance with global env
    def __init__(self, Global = Environment()):
      self.Global = Global

    # Evaluate an expression in the given env
    def evaluate(self, exp, env = None):
      # Set env if needed
      if not env: env = self.Global
      # Data types
      if isNumber(exp):
        return exp
      
      if isString(exp):
        return exp[1:-1]
      # Math operations
      if exp[0] is '+':
        return self.evaluate(exp[1]) + self.evaluate(exp[2])
      
      if exp[0] is '-':
        return self.evaluate(exp[1]) - self.evaluate(exp[2])

      if exp[0] is '*':
        return self.evaluate(exp[1]) * self.evaluate(exp[2])

      if exp[0] is '/':
        return self.evaluate(exp[1]) / self.evaluate(exp[2])
      # Variables
      # Declare
      if exp[0] is 'var':
        [_, name, value] = exp
        return env.define(name, self.evaluate(value))
      # Access
      if isVariableName(exp):
        if len(exp) is 1:
          exp = exp[0]
        return env.lookupVar(exp)
      # Exception
      raise Exception ("Unimplemented  {}".format(exp))



def isNumber(exp):
  if type(exp) is int or type(exp) is float:
    return True
  return False

def isString(exp):
  return type(exp) is str and exp[0] is '"' and exp[-1] is '"'

def isVariableName(exp):
  if len(exp) is 1:
    exp = exp[0]
  variableNameFormat = re.compile("^([a-zA-Z])[a-zA-Z0-9_]*$")
  return type(exp) is str and variableNameFormat.match(exp)

In [273]:
# Tests

abl = Abl(Environment(
    {
        'True'  : True,
        'False' : False,
        'None'  : None,
        'VERSION' : '1.0'
    }
))

# Math
assert abl.evaluate(1) == 1
assert abl.evaluate(1.0) == 1.0
assert abl.evaluate('"Hello World"') == 'Hello World'

assert abl.evaluate(['+', 1, 3]) == 4
assert abl.evaluate(['+', 1.2, 3.3]) == 4.5
assert abl.evaluate(['+', ['+', 1, 4], 4]) == 9
assert abl.evaluate(['-', 4, 3]) == 1
assert abl.evaluate(['*', 6, 3]) == 18
assert abl.evaluate(['/', 9, 3]) == 3

assert abl.evaluate(['+', '"Hello "', '"World"']) == "Hello World"

# Var
assert abl.evaluate(['var', 'x', 10]) == 10
assert abl.evaluate(['x']) == 10
assert abl.evaluate(['var', 'y', 20]) == 20
assert abl.evaluate(['var', 'flag', 'True']) == True
assert abl.evaluate(['var', 'xy', ['*', 'x', 'y']]) == 200
assert abl.evaluate(['VERSION']) == '1.0'

print("All assertions succeded")

All assertions succeded
